In [1]:
%run "../Functions/functions_v1.ipynb"

# Read in Data

# Create ELO Average Win Rate Model

In [ ]:
elo = []
result = []
for x in range(-1200, 1250, 50):
    for y in ["T1", "D", "T2"]:
        elo.append(x)
        result.append(y)

df_base = pd.DataFrame()
df_base["rounded_elo"] = elo
df_base["result_class"] = result
df_base

In [ ]:
df_joined_fin["rounded_elo"] = df_joined_fin["elo_diff"].apply(apply_elo_50_rounding)
df_joined_fin["count"] = 1

counts_by_elo = df_joined_fin.groupby(by = ["rounded_elo"])["count"].sum().reset_index()
counts_by_elo.columns = ["rounded_elo", "total_c"]
results_elo = df_joined_fin.groupby(by = ["rounded_elo", "result_class"])["count"].sum().reset_index()

joined = df_base.merge(results_elo, on = ["rounded_elo", "result_class"], how = "left").merge(counts_by_elo, on = ["rounded_elo"], how = "left").fillna(0)
joined["pct"] = joined["count"] / joined["total_c"]
joined["result_class"] = pd.Categorical(joined["result_class"], categories = ["T1", "D", "T2"], ordered = True)


In [ ]:
sns.lineplot(data = joined[joined["rounded_elo"] >= 0]
             , x = "rounded_elo", y = "pct", hue = "result_class")
#sns.lineplot(data = joined, x = "rounded_elo", y = "pct", hue = "result")
plt.show()

In [ ]:
joined.to_csv(f'../Models/elo_win_rates.csv', index=False)